# Setup

Make sure you have already tried `basics.ipynb`, then follow the cells and instructions below.

In [ ]:
from pybricksdev.connections import BLEPUPConnection, EV3Connection
from asyncio import gather, sleep

car_hub = BLEPUPConnection()
await car_hub.connect('00:16:53:AD:A6:A9')

remote_hub = BLEPUPConnection()
await remote_hub.connect('90:84:2B:4A:8F:7D')

### This script will run on the remote hub with a motor that acts as a dial

For now we just save it. We will run it later.

In [ ]:
%%file build/remote.py

from pybricks.pupdevices import Motor
from pybricks.parameters import Port
from pybricks.tools import wait, StopWatch

dial = Motor(Port.A)

while True:
    
    # In this demo, limit angle to 0--255 for simplicity, so it fits in one byte.
    angle = max(0, min(dial.angle(), 255))
    
    # Print the angle to the PC.
    print(angle)
    wait(10)

### This script will run on the vehicle robot


For now we just save it. We will run it later.

In [ ]:
%%file build/car.py

from pybricks.pupdevices import Motor
from pybricks.parameters import Port, Direction
from pybricks.tools import wait, StopWatch
from pybricks.experimental import getchar


car_motor = Motor(Port.A)

while True:
    # The car listens to characters from the PC.
    char = getchar()
    
    # If we have something, use it to set the duty cycle
    # of the motor, after scaling 255 to 100
    if char is not None:
        car_motor.dc(char * 100 // 255)
        
    wait(5)

### This will run on the PC
For now we just save the function. We will run it later.

In [ ]:
# We can do anything we want with the data, like displaying it.
# In this example, we just send the received number from the
# remote to the car as a single byte

async def forwarder(car_hub, remote_hub):
    
    # Give the hubs some time to start
    await sleep(2)

    while car_hub.state == car_hub.RUNNING or remote_hub.state == remote_hub.RUNNING:
        # Check if the remote has printed anything
        if len(remote_hub.output) > 0:
            
            # If so, let's see what the most recent value is
            line = remote_hub.output[-1]
            
            # In this demo, we are only interested in the most recent value,
            # so we can clear everything else.
            remote_hub.output = []
            
            # Try to convert it to a number and send it as one byte to the receiving hub
            try:
                angle = int(line.decode())
                await car_hub.write(bytes([angle]))
            except:
                pass
        await sleep(0.05)

### Run everything

This runs both robot scripts and the PC script at the same time.

It keeps running until you end both hub scripts using the button.

In [ ]:
await gather(
    remote_hub.run('build/remote.py', print_output=False),
    car_hub.run('build/car.py', print_output=False),
    forwarder(car_hub, remote_hub)
)

In [ ]:
# Disconnect (optional)

await car_hub.disconnect()
await remote_hub.disconnect()